In [1]:
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 
from selenium.webdriver.common.by import By

## ALFACART SCRAPE

In [2]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [7]:
# Scraping the page
def scrape_detail_page(url,category,collection):
    time.sleep(10)
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.alfacart
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]

    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    result = pd.DataFrame()
    while(True):
        path = driver.find_elements_by_css_selector('.item')
        try:
            for x in range(0,len(path)):
                discount_price = re.sub("\.","",re.sub("Rp ","",path[x].find_element_by_css_selector('.lead').text))
                origin_price = re.sub("\.","",re.sub("Rp ","",path[x].find_element_by_css_selector('.fprice').text))
                if(origin_price==''):
                    origin_price = discount_price
                name = path[x].find_elements_by_css_selector('.ga-click-product-category-sub')[1].text
                image = path[x].find_element_by_css_selector('img').get_attribute('src')
                sku = image

                time_check = 0
                while(image is None and time_check < 60):
                    time_check = time_check+1
                    path[x].find_element_by_css_selector('.title').location_once_scrolled_into_view
                    time.sleep(1)
                    image = path[x].find_element_by_css_selector('.wrp-media > img').get_attribute('src')
                    print("wait_image" + " " + name)

                # Return result
                date = datetime.datetime.now().strftime("%Y-%m-%d")
                new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':origin_price, 'discount_price':discount_price, 'image':image,'category':category,'scraping_date':date}])
                result = result.append(new_result)

                # save to mongodb
                collection.insert_many(new_result.to_dict('records'))
                
            # next page
            page =  driver.find_elements_by_css_selector('#paging > li')
            page[len(page)-1].find_element_by_css_selector('a').click()
        except:
            PrintException()
            break
    driver.close()
    return result


In [11]:
# MAIN
alfacart_scrape = pd.read_csv('..\\Data\\alfacart_category.csv',sep=",")
result = pd.DataFrame()
for x in range(0,len(alfacart_scrape)):
    category = alfacart_scrape.iloc[x]
    print(str(x) + "/" + str(len(alfacart_scrape)) + " " + category['collection'] + " - " + category['category'])
    
    url = category['link']
    collection = category['collection']
    
    result = scrape_detail_page(url,category['category'],collection)
    print(result.head())

0/64 makanan_segar_beku_kaleng - buah--sayuran-segar
Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

              category discount_price  \
0  buah--sayuran-segar          55080   

                                               image  \
0  https://cs1.alfacart.com/product/2124684/21246...   

                              name origin_price scraping_date  \
0  Kurma Tunisia Palm Fruit 500 gr        55080    2019-09-25   

                                                 sku  
0  https://cs1.alfacart.com/product/2124684/21246...  
1/64 makanan_segar_beku_kaleng - makanan-beku-olahan
Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

              category discount_price  \
0 

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

         category discount_price  \
0  minuman-instan          57000   
0  minuman-instan          12900   
0  minuman-instan           8600   
0  minuman-instan          11700   
0  minuman-instan          10900   

                                               image  \
0  https://cs1.alfacart.com/product/1033143/_s_e_...   
0  https://cs1.alfacart.com/product/1/_4_2_42771_...   
0  https://cs1.alfacart.com/product/1/A1212002203...   
0  https://cs1.alfacart.com/product/1/_1_2_125516...   
0  https://cs1.alfacart.com/product/1/1_A12120004...   

                                        name origin_price scraping_date  \
0              Indocafe Original Blend 200Gr        57000    2019-09-25   
0                Luwak White Koffie 10 x 20g        12900    2019-09-25   
0  TOP Coffee 

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

  category discount_price                                              image  \
0    beras          80000  https://cs1.alfacart.com/product/1033143/_2_8_...   
0    beras         280000  https://cs1.alfacart.com/product/1033143/_a_0_...   
0    beras          31900  https://cs1.alfacart.com/product/1/_02Mar17b_A...   
0    beras          64000  https://cs1.alfacart.com/product/1/_ImageJSM28...   
0    beras          64000  https://cs1.alfacart.com/product/1/1_A68775600...   

                                  name origin_price scraping_date  \
0   Beras Cap Ikan Mas Setra Ramos 5kg        80000    2019-09-25   
0  Beras Cap Ikan Mas Setra Ramos 20KG       280000    2019-09-25   
0             Alfamart Beras Merah 2kg        31900    2019-09-25   
0       Alfamart Beras Setra Ramos 5

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

        category discount_price  \
0  bumbu-masakan          68500   
0  bumbu-masakan           1700   
0  bumbu-masakan          14300   
0  bumbu-masakan           5000   
0  bumbu-masakan          10200   

                                               image  \
0  https://cs1.alfacart.com/product/1033143/_h_e_...   
0  https://cs1.alfacart.com/product/1/1_A09130277...   
0  https://cs1.alfacart.com/product/1/_15jun17z_A...   
0  https://cs1.alfacart.com/product/1/1_A09130277...   
0  https://cs1.alfacart.com/product/1/_a_b_abc-sa...   

                                name origin_price scraping_date  \
0           Heinz White Vinegar 32oz        68500    2019-09-25   
0   Racik Bumbu Nasi Goreng Ayam 20G         1900    2019-09-25   
0  LEE KUM KEE Saus Tiram Botol 145g        

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

            category discount_price  \
0  puding--agar-agar           7900   
0  puding--agar-agar           7900   
0  puding--agar-agar           6000   
0  puding--agar-agar           6000   
0  puding--agar-agar           7200   

                                               image  \
0  https://cs1.alfacart.com/product/1/_1_2_125073...   
0  https://cs1.alfacart.com/product/1/_6_2_620029...   
0  https://cs1.alfacart.com/product/1/1_A10280000...   
0  https://cs1.alfacart.com/product/1/1_A10280002...   
0  https://cs1.alfacart.com/product/1/_1_1_118345...   

                                 name origin_price scraping_date  \
0                Pino Ice Cup 6 x 85g         7900    2019-09-25   
0          Alfamart Puding Buah 3X85G         7900    2019-09-25   
0        YEKO Pud

...

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

         category discount_price  \
0  selai--taburan          21000   
0  selai--taburan           9900   
0  selai--taburan          14900   
0  selai--taburan          16900   
0  selai--taburan           6200   

                                               image  \
0  https://cs1.alfacart.com/product/1033143/_4_9_...   
0  https://cs1.alfacart.com/product/1/ASS1_small.jpg   
0  https://cs1.alfacart.com/product/1/_14Des16RF4...   
0  https://cs1.alfacart.com/product/1/_14Des16RF4...   
0  https://cs1.alfacart.com/product/1/_1_2_121564...   

                                   name origin_price scraping_date  \
0  Ceres Hagelslag Family Classic 225Gr        21000    2019-09-25   
0      Alfamart Selai Stroberi Cup 150g         9900    2019-09-25   
0       Alfamart Selai Cokela

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

            category discount_price  \
0  minuman-kesehatan           6500   
0  minuman-kesehatan           6300   
0  minuman-kesehatan           6500   
0  minuman-kesehatan           6900   
0  minuman-kesehatan           6900   

                                               image  \
0  https://cs1.alfacart.com/product/1/_17jan17a_A...   
0  https://cs1.alfacart.com/product/1/A1274000439...   
0  https://cs1.alfacart.com/product/1/_TakedownPD...   
0  https://cs1.alfacart.com/product/1/_PDMori15De...   
0  https://cs1.alfacart.com/product/1/_PDMori15De...   

                                     name origin_price scraping_date  \
0  KINO Larutan Penyegar Cap Kaki Tiga...         6500    2019-09-25   
0        Kiranti Sehat Datang Bulan 150ml         6300    2019-09-25   
0    

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

          category discount_price  \
0  susu-6-12-bulan         104400   
0  susu-6-12-bulan         145900   
0  susu-6-12-bulan         139500   
0  susu-6-12-bulan          44300   
0  susu-6-12-bulan          22700   

                                               image  \
0  https://cs1.alfacart.com/product/1/A0677004784...   
0  https://cs1.alfacart.com/product/1/A0677004784...   
0  https://cs1.alfacart.com/product/1/_10feb17a_A...   
0  https://cs1.alfacart.com/product/1/_1_0_102739...   
0  https://cs1.alfacart.com/product/1/A0679030360...   

                                    name origin_price scraping_date  \
0              S-26 Promil 2 Vanila 400g       104400    2019-09-25   
0         S-26 Promil Gold 2 Kaleng 400g       145900    2019-09-25   
0  morinaga Chil-Mil

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

      category discount_price  \
0  susu-dewasa         122500   
0  susu-dewasa          36900   
0  susu-dewasa          19900   
0  susu-dewasa          38700   
0  susu-dewasa          39400   

                                               image  \
0  https://cs1.alfacart.com/product/1/_a_1_a13070...   
0  https://cs1.alfacart.com/product/1/A1307000463...   
0  https://cs1.alfacart.com/product/1/194645-1_sm...   
0  https://cs1.alfacart.com/product/1/193826-1_sm...   
0  https://cs1.alfacart.com/product/1/A1307000128...   

                                       name origin_price scraping_date  \
0                    Ensure Vanila Can 400g       122500    2019-09-25   
0         Zee Swizz Chocolate Milk 10 x 40g        36900    2019-09-25   
0   Dancow Fortigo Instant Cokelat 

Connected successfully!!!
EXCEPTION IN (<ipython-input-7-2f9b671c2f27>, LINE 50 "page[len(page)-1].find_element_by_css_selector('a').click()"): Message: Element <a href="#"> could not be scrolled into view

        category discount_price  \
0  makanan-impor         950000   
0  makanan-impor          56100   
0  makanan-impor          32640   
0  makanan-impor          32640   
0  makanan-impor          32640   

                                               image  \
0  https://cs1.alfacart.com/product/1499949/sukka...   
0  https://cs1.alfacart.com/product/2124684/21246...   
0  https://cs1.alfacart.com/product/2124684/21246...   
0  https://cs1.alfacart.com/product/2124684/21246...   
0  https://cs1.alfacart.com/product/2124684/21246...   

                                    name origin_price scraping_date  \
0  Premiumdates Kurma Sukkari 5 kilogram       950000    2019-09-25   
0                     Kinder Schoko Bons        56100    2019-09-25   
0       Xiaomimi Tian Tian Jue B

In [9]:
result

,category,discount_price,image,name,origin_price,scraping_date,sku
0,minuman-kesehatan,6500,https://cs1.alfacart.com/product/1/_17jan17a_A...,KINO Larutan Penyegar Cap Kaki Tiga...,6500,2019-09-25,https://cs1.alfacart.com/product/1/_17jan17a_A...
0,minuman-kesehatan,6300,https://cs1.alfacart.com/product/1/A1274000439...,Kiranti Sehat Datang Bulan 150ml,6300,2019-09-25,https://cs1.alfacart.com/product/1/A1274000439...
0,minuman-kesehatan,6500,https://cs1.alfacart.com/product/1/_TakedownPD...,KINO Larutan Cap Kaki Tiga Leci,6500,2019-09-25,https://cs1.alfacart.com/product/1/_TakedownPD...
0,minuman-kesehatan,6900,https://cs1.alfacart.com/product/1/_PDMori15De...,SINDE Larutan Strawberry 320ml,6900,2019-09-25,https://cs1.alfacart.com/product/1/_PDMori15De...
0,minuman-kesehatan,6900,https://cs1.alfacart.com/product/1/_PDMori15De...,SINDE Larutan Penyegar Orange 320ml,6900,2019-09-25,https://cs1.alfacart.com/product/1/_PDMori15De...
0,minuman-kesehatan,6500,https://cs1.alfacart.com/product/4/_OKE_A12740...,LASEGAR Minuman Penyegar Jeruk Nipis...,6500,2019-09-25,https://cs1.alfacart.com/product/4/_OKE_A12740...
0,minuman-kesehatan,6500,https://cs1.alfacart.com/product/1/_OKE_A12740...,Cap Kaki Tiga Larutan Penyegar Leci...,6500,2019-09-25,https://cs1.alfacart.com/product/1/_OKE_A12740...
0,minuman-kesehatan,6500,https://cs1.alfacart.com/product/1/_OKE_A12740...,Cap Kaki Tiga Larutan Penyegar Jambu...,6500,2019-09-25,https://cs1.alfacart.com/product/1/_OKE_A12740...
0,minuman-kesehatan,6500,https://cs1.alfacart.com/product/1/_OKE_A12740...,Cap Kaki Tiga Larutan Penyegar Jeruk...,6500,2019-09-25,https://cs1.alfacart.com/product/1/_OKE_A12740...
0,minuman-kesehatan,7900,https://cs1.alfacart.com/product/1/_ImageOrigi...,Nu OCEANA Lemonade Pet 460ml,7900,2019-09-25,https://cs1.alfacart.com/product/1/_ImageOrigi...


## After Scraping

In [36]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db_out = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db_out[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if '_id' in df.columns:
        del df['_id']
    
    if db == 'visenze_tagging':
        del df['created_date'], df['metadata'], df['box']
    return df

In [37]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

dbname = 'alfacart'
mydb = myclient[dbname]
result = pd.DataFrame()

#list the collections
for coll in mydb.collection_names():
    data = read_mongo(dbname,coll)
    data['collection'] = coll
    data = data
    result = result.append(data)

result = result.drop_duplicates(['name','image'])
filename = "ALFAMART"+str(datetime.datetime.now().strftime("%Y%m%d"))+'.csv'
result.to_csv('..\\Result\\'+filename,index=False)